<a href="https://colab.research.google.com/github/shachi-i/movie-recommendation-system/blob/main/Movie_recommenation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importing dependencies

In [ ]:
import numpy as np
import pandas as pd
import difflib #difflib is used to find the closest match of the input value to the dataset incase theres spelling errors
from sklearn.feature_extraction.text import TfidfVectorizer #convert the textual data to numerical data
from sklearn.metrics.pairwise import cosine_similarity #assign similarity score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

Data collection and preprocessing

In [ ]:
#loading data from csv file to a pandas dataframe
movies_data = pd.read_csv('/content/archive.zip')

In [ ]:
#printing the first five rows of the dataframe
movies_data.head()

,Name,Aired Date,Year of release,Original Network,Aired On,Number of Episodes,Duration,Content Rating,Rating,Synopsis,Genre,Tags,Director,Screenwriter,Cast,Production companies,Rank
0,Move to Heaven,"May 14, 2021",2021,Netflix,Friday,10,52 min.,18+ Restricted (violence & profanity),9.2,Geu Roo is a young autistic man. He works for ...,"Life, Drama, Family","Autism, Uncle-Nephew Relationship, Death, Sava...",Kim Sung Ho,Yoon Ji Ryun,"Lee Je Hoon, Tang Jun Sang, Hong Seung Hee, Ju...","Page One Film, Number Three Pictures",#1
1,Flower of Evil,"Jul 29, 2020 - Sep 23, 2020",2020,tvN,"Wednesday, Thursday",16,1 hr. 10 min.,15+ - Teens 15 or older,9.1,Although Baek Hee Sung is hiding a dark secret...,"Thriller, Romance, Crime, Melodrama","Married Couple, Deception, Suspense, Family Se...","Kim Chul Gyu, Yoon Jong Ho",Yoo Jung Hee,"Lee Joon Gi, Moon Chae Won, Jang Hee Jin, Seo ...",Monster Union,#2
2,Hospital Playlist,"Mar 12, 2020 - May 28, 2020",2020,"Netflix, tvN",Thursday,12,1 hr. 30 min.,15+ - Teens 15 or older,9.1,The stories of people going through their days...,"Friendship, Romance, Life, Medical","Strong Friendship, Multiple Mains, Best Friend...",Shin Won Ho,Lee Woo Jung,"Jo Jung Suk, Yoo Yeon Seok, Jung Kyung Ho, Kim...","Egg Is Coming, CJ ENM",#3
3,Hospital Playlist 2,"Jun 17, 2021 - Sep 16, 2021",2021,"Netflix, tvN",Thursday,12,1 hr. 40 min.,15+ - Teens 15 or older,9.1,Everyday is extraordinary for five doctors and...,"Friendship, Romance, Life, Medical","Workplace, Strong Friendship, Best Friends, Mu...",Shin Won Ho,Lee Woo Jung,"Jo Jung Suk, Yoo Yeon Seok, Jung Kyung Ho, Kim...","Egg Is Coming, CJ ENM",#4
4,My Mister,"Mar 21, 2018 - May 17, 2018",2018,tvN,"Wednesday, Thursday",16,1 hr. 17 min.,15+ - Teens 15 or older,9.1,Park Dong Hoon is a middle-aged engineer who i...,"Psychological, Life, Drama, Family","Age Gap, Nice Male Lead, Strong Female Lead, H...","Kim Won Suk, Kim Sang Woo",Park Hae Young,"Lee Sun Kyun, IU, Park Ho San, Song Sae Byuk, ...",Chorokbaem Media,#5


In [ ]:
#number of rows and columns
movies_data.shape

(250, 17)

In [ ]:
#selecting relevant features for recommendation
selected_features= ['Name','Synopsis','Genre','Tags','Cast'	 ]
print(selected_features)

['Name', 'Synopsis', 'Genre', 'Tags', 'Cast']


In [ ]:
#replacing null calues with null string
for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

In [ ]:
#combining all the 5 selected features
combined_features = movies_data['Name']+' '+movies_data['Synopsis']+' '+movies_data['Genre']+' '+movies_data['Tags']+' '+movies_data['Cast']

In [ ]:
print(combined_features)

0      Move to Heaven Geu Roo is a young autistic man...
1      Flower of Evil Although Baek Hee Sung is hidin...
2      Hospital Playlist The stories of people going ...
3      Hospital Playlist 2 Everyday is extraordinary ...
4      My Mister Park Dong Hoon is a middle-aged engi...
                             ...                        
245    Live Up to Your Name Heo Im, who is born in a ...
246    Queen for Seven Days This drama is about the l...
247    Memory Park Tae Suk is a lawyer who finds out ...
248    A Korean Odyssey In 2017, Son Oh Gong and Ma W...
249    Voice 4: Judgment Hour A serial killer with an...
Length: 250, dtype: object


In [ ]:
#converting test data to feature vectors or numerical values
vectorizer=TfidfVectorizer()


In [ ]:
feature_vectors=vectorizer.fit_transform(combined_features)

In [ ]:
print(feature_vectors)

  (0, 3612)	0.07457228756513246
  (0, 2617)	0.05917244307681523
  (0, 2230)	0.04188626334625787
  (0, 4491)	0.06961573064600823
  (0, 3887)	0.050687424497402066
  (0, 2247)	0.0625741071317026
  (0, 1911)	0.035885187433744165
  (0, 3582)	0.047626104544689185
  (0, 1978)	0.058406570912936606
  (0, 2246)	0.056969965262844056
  (0, 3971)	0.09350059371373862
  (0, 1981)	0.04975588483835737
  (0, 2207)	0.08155816217987343
  (0, 2352)	0.048705992787856124
  (0, 2829)	0.1004864683284796
  (0, 228)	0.01865992451414187
  (0, 745)	0.1004864683284796
  (0, 878)	0.08155816217987343
  (0, 1418)	0.06823665188324939
  (0, 2360)	0.08155816217987343
  (0, 3719)	0.04573086757827785
  (0, 3986)	0.07111487461068042
  (0, 2631)	0.1004864683284796
  (0, 3943)	0.08854403679461441
  (0, 3473)	0.09350059371373862
  :	:
  (249, 3693)	0.04262490956740272
  (249, 3925)	0.06645713290966217
  (249, 934)	0.05852020130050163
  (249, 4053)	0.04262490956740272
  (249, 1930)	0.037420907062651686
  (249, 1655)	0.072688392

In [ ]:
#getting the similarity scores using cosine similarity
similarity=cosine_similarity(feature_vectors)

In [ ]:
print(similarity)

[[1.         0.07560855 0.04983295 ... 0.10925479 0.05485098 0.05094272]
 [0.07560855 1.         0.03704071 ... 0.09841493 0.12668175 0.11544109]
 [0.04983295 0.03704071 1.         ... 0.07805202 0.08459789 0.06370909]
 ...
 [0.10925479 0.09841493 0.07805202 ... 1.         0.07094918 0.06015598]
 [0.05485098 0.12668175 0.08459789 ... 0.07094918 1.         0.08537471]
 [0.05094272 0.11544109 0.06370909 ... 0.06015598 0.08537471 1.        ]]


In [ ]:
print(similarity.shape)

(250, 250)


In [ ]:
#getting the movie name from the user
kdrama_name=input('Enter your favourite kdrama name: ')

KeyboardInterrupt: Interrupted by user

In [ ]:
#creating a list with all the kdrama names given in the dataset
list_of_all_kdrama_names=movies_data['Name'].tolist()
print(list_of_all_kdrama_names)

['Move to Heaven', 'Flower of Evil', 'Hospital Playlist', 'Hospital Playlist 2', 'My Mister', 'Reply 1988', 'Weak Hero Class 1', 'Prison Playbook', 'Alchemy of Souls', 'Extraordinary Attorney Woo', 'Mr. Queen', 'Mother', "It's Okay to Not Be Okay", 'Crash Landing on You', 'Vincenzo', 'Navillera', 'Signal', 'Mr. Sunshine', 'Happiness', 'Kingdom: Season 2', 'SKY Castle', 'Tomorrow', 'Healer', 'Stranger', 'Twenty-Five Twenty-One', 'The Red Sleeve', 'Goblin', 'The Uncanny Counter', 'Mouse', 'Kingdom', 'Weightlifting Fairy Kim Bok Joo', 'D.P.', 'The Devil Judge', 'The Penthouse', 'Youth of May', 'Taxi Driver', 'Life on Mars', 'Beyond Evil', 'Racket Boys', 'Hometown Cha-Cha-Cha', 'Six Flying Dragons', 'Our Beloved Summer', 'The Guest', 'Dear My Friends', 'While You Were Sleeping', 'The Penthouse 2', 'Chicago Typewriter', '18 Again', 'Arthdal Chronicles Part 2', 'Arthdal Chronicles Part 3', 'Through the Darkness', 'Dr. Romantic 2', 'Defendant', 'Our Blues', 'Sweet Home', 'Kill Me, Heal Me', '

In [ ]:
#finding the close match for the kdrama given by the user
find_close_match=difflib.get_close_matches(kdrama_name,list_of_all_kdrama_names)
print(find_close_match)

['Sweet Home']


In [ ]:
close_match=find_close_match[0]
print(close_match)

Sweet Home


In [ ]:
#find the index of the movie with title
index_of_the_kdrama=movies_data[movies_data.Name==close_match].index.values[0]
print(index_of_the_kdrama)

54


In [ ]:
#getting a list of similar movies
similarity_score=list(enumerate(similarity[index_of_the_kdrama]))
print(similarity_score) #displays score of all movies

[(0, 0.05521102159661963), (1, 0.08979451090948994), (2, 0.0775923862030669), (3, 0.06623462285987412), (4, 0.08991695224898574), (5, 0.11662815214164002), (6, 0.140752064352243), (7, 0.0832906756451825), (8, 0.09030802151389397), (9, 0.056494659225198106), (10, 0.06616211452811169), (11, 0.07993297141736044), (12, 0.05142251803576113), (13, 0.0859153516084409), (14, 0.06638886463069943), (15, 0.07658385491090339), (16, 0.09717790115414358), (17, 0.0787552720235437), (18, 0.17370851662259523), (19, 0.12344881960189408), (20, 0.05742868173148355), (21, 0.10651618960258084), (22, 0.07617835594297488), (23, 0.03804781661746712), (24, 0.08876381262306766), (25, 0.08925969608553032), (26, 0.08368683808412855), (27, 0.09258598323927589), (28, 0.10753922937957722), (29, 0.10426073734198679), (30, 0.0560067363887517), (31, 0.06246904006776736), (32, 0.07324802494682175), (33, 0.12428106273766372), (34, 0.12968508709230525), (35, 0.06313781013105042), (36, 0.07505688312697764), (37, 0.064812161

In [ ]:
len(similarity_score)

250

In [ ]:
#sorting movies based on similarity score
sorted_similar_kdrama=sorted(similarity_score,key=lambda x:x[1],reverse=True)
print(sorted_similar_kdrama)

[(54, 1.0), (67, 0.2490635106154907), (18, 0.17370851662259523), (97, 0.16894607963485542), (135, 0.1675366267642431), (103, 0.15439546596530015), (80, 0.1424672490416617), (6, 0.140752064352243), (136, 0.13684679191616475), (151, 0.13379973939306622), (56, 0.13084903093178085), (34, 0.12968508709230525), (232, 0.12771757215094887), (185, 0.12739257092045794), (109, 0.12605105223583218), (116, 0.12479465105033), (33, 0.12428106273766372), (105, 0.12413215839141083), (19, 0.12344881960189408), (157, 0.12237337001754969), (179, 0.12006835226815071), (221, 0.11962507709799003), (228, 0.11940039387919343), (208, 0.11934051615198488), (148, 0.11789611441448161), (101, 0.11771926329950091), (175, 0.11754989432443756), (106, 0.11752990599593709), (181, 0.11750717855803838), (69, 0.11692717960936358), (212, 0.11676636323814027), (5, 0.11662815214164002), (118, 0.11655057366979278), (199, 0.11615414187811009), (234, 0.1159978304576997), (204, 0.11592971696161648), (149, 0.11537736761816987), (1

In [ ]:
#print the name of similar movies based on the index
print('Kdrama suggested for you: \n')

i=1
for kdrama in sorted_similar_kdrama:
  index=kdrama[0]
  title_from_index=movies_data[movies_data.index==index]['Name'].values[0]
  if(i<30):
    print(i,'.',title_from_index)
    i+=1

Kdrama suggested for you: 

1 . Sweet Home
2 . Strangers from Hell
3 . Happiness
4 . 365
5 . All of Us Are Dead
6 . Beautiful World
7 . The King of Pigs
8 . Weak Hero Class 1
9 . Extracurricular
10 . Train
11 . Moon Lovers
12 . Youth of May
13 . 49 Days
14 . Class of Lies
15 . One Ordinary Day
16 . W
17 . The Penthouse
18 . Kairos
19 . Kingdom: Season 2
20 . Extraordinary You
21 . Tale of the Nine-Tailed
22 . Mine
23 . Light on Me
24 . Again My Life
25 . From Now On, Showtime!
26 . Nobody Knows
27 . Seasons of Blossom
28 . I Hear Your Voice
29 . He Is Psychometric


Kdrama recommendation system


In [ ]:
kdrama_name=input('Enter your favourite kdrama name: ')

list_of_all_kdrama_names=movies_data['Name'].tolist()

find_close_match=difflib.get_close_matches(kdrama_name,list_of_all_kdrama_names)

close_match=find_close_match[0]

index_of_the_kdrama=movies_data[movies_data.Name==close_match].index.values[0]

similarity_score=list(enumerate(similarity[index_of_the_kdrama]))

sorted_similar_kdrama=sorted(similarity_score,key=lambda x:x[1],reverse=True)

print('Kdrama suggested for you: \n')

i=1
for kdrama in sorted_similar_kdrama:
  index=kdrama[0]
  title_from_index=movies_data[movies_data.index==index]['Name'].values[0]
  if(i<30):
    print(i,'.',title_from_index)
    i+=1


Enter your favourite kdrama name: ctash landing on you
Kdrama suggested for you: 

1 . Crash Landing on You
2 . Snowdrop
3 . The Hymn of Death
4 . Mr. Queen
5 . Twenty-Five Twenty-One
6 . Mr. Sunshine
7 . Graceful Family
8 . My Love from the Star
9 . Descendants of the Sun
10 . Extraordinary You
11 . Mad Dog
12 . Chicago Typewriter
13 . Kairos
14 . Military Prosecutor Doberman
15 . Angel's Last Mission: Love
16 . Money Heist: Korea - Joint Economic Area - Part 1
17 . The Beauty Inside
18 . Thirty But Seventeen
19 . One Spring Night
20 . 49 Days
21 . Fight For My Way
22 . My Unfamiliar Family
23 . Kill Me, Heal Me
24 . Mine
25 . 365
26 . A Business Proposal
27 . Just Between Lovers
28 . D.P.
29 . Awaken
